## Dependencies

In [13]:
import pymysql
pymysql.install_as_MySQLdb()

In [14]:
import pandas as pd
from sqlalchemy import create_engine
import json
from pprint import pprint
import requests

## Import Files

In [15]:
#Zipcode Lat/Lon
zipcodes_file = "zipcodes.csv"
zips_df = pd.read_csv(zipcodes_file)
zips_df.head()

#State Lat/Lon
states_file = "states.csv"
states_df = pd.read_csv(states_file)
states_df.head()

# Metric Data
## Anemia
anem_file = "Anemia Management.csv"
anem_df_1 = pd.read_csv(anem_file)
anem_df_1.head()

# Depression
depr_file = "Depression Screening.csv"
depr_df_1 = pd.read_csv(depr_file)
depr_df_1.head()

,Facility Name,CMS Certification Number (CCN),Alternate CCN 1,Address 1,Address 2,City,State,Zipcode,Network,Measure Name,Clinical Depression Screening and Follow-up Measure Score,State Avg Clinical Depression Screening and Follow-up Measure Score,National Avg Clinical Depression Screening and Follow-up Measure Score
0,CHILDRENS HOSPITAL DIALYSIS,12306,13300,1600 7TH AVENUE SOUTH,-,BIRMINGHAM,AL,35233,8,CLINICAL DEPRESSION,10,10,10
1,FMC CAPITOL CITY,12500,-,255 S JACKSON STREET,-,MONTGOMERY,AL,36104,8,CLINICAL DEPRESSION,10,10,10
2,GADSDEN DIALYSIS,12501,-,409 SOUTH FIRST STREET,-,GADSDEN,AL,35901,8,CLINICAL DEPRESSION,10,10,10
3,TUSCALOOSA UNIVERSITY DIALYSIS,12502,-,220 15TH STREET,-,TUSCALOOSA,AL,35401,8,CLINICAL DEPRESSION,10,10,10
4,PCD MONTGOMERY,12505,-,1001 FOREST AVENUE,-,MONTGOMERY,AL,36106,8,CLINICAL DEPRESSION,10,10,10


## Transform Data

In [18]:
# ANEMIA

# Merge Metric & Zipcode Lat/Lon Data
anem_merged_df = pd.merge(anem_df_1, zips_df, how = "left", on="Zipcode")
anem_merged_df = anem_merged_df[["Facility Name", "CMS Certification Number (CCN)", "City_x", "State_x", "Zipcode", "Measure Name", "Anemia Management Measure Score", "State Avg Anemia Management Measure Score", "National Avg Anemia Management Measure Score", "Lat", "Long"]]
anem_merged_df = anem_merged_df.rename(columns = {"Lat": "ZipLat", "Long" : "ZipLon", "State_x" : "State", "City_x" : "City"})
anem_merged_df.head()

# Merge in State Lat/Lon data
anem_df = pd.merge(anem_merged_df, states_df, how = "left", on="State")
anem_df = anem_df.drop(columns=["City_y"])
anem_df = anem_df[anem_df["Anemia Management Measure Score"] != "No Score"]
anem_df = anem_df.rename(columns = {"Facility Name" : "FacilityName", "CMS Certification Number (CCN)" : "CCN", "City_x" : "City", "Measure Name" : "MeasureName", "Anemia Management Measure Score" : "AnemiaScore", "State Avg Anemia Management Measure Score" : "StateAvg", "National Avg Anemia Management Measure Score" : "NationalAvg",  "Latitude" : "StateLat", "Longitude" : "StateLon"})
anem_df.head()

# State avgs 
anem_stateavgs_df = anem_df.groupby(["State"]).mean()
anem_stateavgs_df = anem_stateavgs_df[["StateAvg", "NationalAvg", "StateLat", "StateLon"]]
anem_stateavgs_df = anem_stateavgs_df.dropna()
anem_stateavgs_df

#------------------------------------------
# DEPRESSION

# Merge Metric & Zipcode Lat/Lon Data
depr_merged_df = pd.merge(depr_df_1, zips_df, how = "left", on="Zipcode")
depr_merged_df = depr_merged_df[["Facility Name", "CMS Certification Number (CCN)", "City_x", "State_x", "Zipcode", "Measure Name", "Clinical Depression Screening and Follow-up Measure Score", "State Avg Clinical Depression Screening and Follow-up Measure Score", "National Avg Clinical Depression Screening and Follow-up Measure Score", "Lat", "Long"]]
depr_merged_df = depr_merged_df.rename(columns = {"Lat": "ZipLat", "Long" : "ZipLon", "State_x" : "State", "City_x" : "City"})
depr_merged_df.head()

# Merge in State Lat/Lon data
depr_df = pd.merge(depr_merged_df, states_df, how = "left", on="State")
depr_df = depr_df.drop(columns=["City_y"])
depr_df = depr_df[depr_df["Clinical Depression Screening and Follow-up Measure Score"] != "No Score"]
depr_df = depr_df.rename(columns = {"Facility Name" : "FacilityName", "CMS Certification Number (CCN)" : "CCN", "City_x" : "City", "Measure Name" : "MeasureName", "Clinical Depression Screening and Follow-up Measure Score" : "DepressionScore", "State Avg Clinical Depression Screening and Follow-up Measure Score" : "StateAvg", "National Avg Clinical Depression Screening and Follow-up Measure Score" : "NationalAvg",  "Latitude" : "StateLat", "Longitude" : "StateLon"})
depr_df.head()

# State avgs 
depr_stateavgs_df = depr_df.groupby(["State"]).mean()
depr_stateavgs_df = depr_stateavgs_df[["StateAvg", "NationalAvg", "StateLat", "StateLon"]]
depr_stateavgs_df = depr_stateavgs_df.dropna()
depr_stateavgs_df

,StateAvg,NationalAvg,StateLat,StateLon
State,,,,
AK,10.0,10.0,61.302501,-158.775020
AL,10.0,10.0,32.601011,-86.680737
AR,10.0,10.0,34.751927,-92.131378
AZ,10.0,10.0,34.168218,-111.930907
CA,10.0,10.0,37.271874,-119.270415
CO,10.0,10.0,38.997934,-105.550567
CT,10.0,10.0,41.518783,-72.757507
DC,10.0,10.0,38.899349,-77.014567
DE,9.0,10.0,39.145251,-75.418921


## Database

In [19]:
connection_string = "root:toor2@localhost/esrd_db"
engine = create_engine(f'mysql://{connection_string}')

In [25]:
# Confirm tables
engine.table_names()

['anem_stateavgs_tbl', 'anem_tbl', 'depr_stateavgs_tbl', 'depr_tbl']

In [26]:
anem_df.to_sql(name='anem_tbl', con=engine, if_exists='append', index=False)

In [27]:
anem_stateavgs_df.to_sql(name='anem_stateavgs_tbl', con=engine, if_exists='append', index=False)

In [28]:
depr_df.to_sql(name='depr_tbl', con=engine, if_exists='append', index=False)

In [29]:
depr_stateavgs_df.to_sql(name='depr_stateavgs_tbl', con=engine, if_exists='append', index=False)